In [ ]:
# imports for downloading files
import urllib.request
import zipfile
import os

#  imports for the network
import torchvision, torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import time

In [3]:
import os
from PIL import Image
from sklearn.model_selection import train_test_split
folder_path = "/home/xurxo/Documentos/Asignaturas Master/Semestre_1/Deep_Learning/practicas/project/Dataset/Segmentation"

image_path = folder_path + "/Images"
mask_path = folder_path + "/Masks"

"""
Creation of the dataset: It comprises
two folders, one for the images and one for the masks.
The idea is to create a map between the images and the masks, 
then splitting into train, validation and testing
"""


image_files = os.listdir(image_path)
mask_files = os.listdir(mask_path)

image_mapping = {}

for image_file in image_files:
    image_name = os.path.splitext(image_file)[0]
    mask_file = image_name + ".png"
    
    if mask_file in mask_files:
        image_mapping[image_file] = mask_file

print(image_mapping)

#now we split the dataset into train(70), validation(15) and test (15)

# Split the image_mapping into train, validation, and test subsets
train_mapping, test_mapping = train_test_split(list(image_mapping.items()), test_size=0.3, random_state=42)
validation_mapping, test_mapping = train_test_split(test_mapping, test_size=0.5, random_state=42)

# Convert the train, validation, and test mappings back to dictionaries
train_mapping = dict(train_mapping)
validation_mapping = dict(validation_mapping)
test_mapping = dict(test_mapping)

# Print the train, validation, and test mappings
print("Train Mapping:")
print(train_mapping)
print("Validation Mapping:")
print(validation_mapping)
print("Test Mapping:")
print(test_mapping)





{'image_1359.jpg': 'image_1359.png', 'image_785.jpg': 'image_785.png', 'image_1285.jpg': 'image_1285.png', 'image_1857.jpg': 'image_1857.png', 'image_1604.jpg': 'image_1604.png', 'image_1373.jpg': 'image_1373.png', 'image_578.jpg': 'image_578.png', 'image_1578.jpg': 'image_1578.png', 'image_930.jpg': 'image_930.png', 'image_25.jpg': 'image_25.png', 'image_1713.jpg': 'image_1713.png', 'image_1135.jpg': 'image_1135.png', 'image_330.jpg': 'image_330.png', 'image_953.jpg': 'image_953.png', 'image_1968.jpg': 'image_1968.png', 'image_1970.jpg': 'image_1970.png', 'image_124.jpg': 'image_124.png', 'image_1000.jpg': 'image_1000.png', 'image_564.jpg': 'image_564.png', 'image_56.jpg': 'image_56.png', 'image_1533.jpg': 'image_1533.png', 'image_988.jpg': 'image_988.png', 'image_975.jpg': 'image_975.png', 'image_1431.jpg': 'image_1431.png', 'image_1088.jpg': 'image_1088.png', 'image_1757.jpg': 'image_1757.png', 'image_1317.jpg': 'image_1317.png', 'image_1393.jpg': 'image_1393.png', 'image_1678.jpg':